<a href="https://colab.research.google.com/github/gazal-gupta/Deep_Learning/blob/master/Univariate_Time_Series_InClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Univariate Time Series
```



In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import math
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense,Activation, Dropout
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
import seaborn as sns

In [59]:
!wget https://www.dropbox.com/sh/edjj4duz16m6fdj/AADxEIKHLlB38oQHMBsEeNJoa/international-airline-passengers.csv

--2019-12-22 06:07:46--  https://www.dropbox.com/sh/edjj4duz16m6fdj/AADxEIKHLlB38oQHMBsEeNJoa/international-airline-passengers.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/edjj4duz16m6fdj/AADxEIKHLlB38oQHMBsEeNJoa/international-airline-passengers.csv [following]
--2019-12-22 06:07:46--  https://www.dropbox.com/sh/raw/edjj4duz16m6fdj/AADxEIKHLlB38oQHMBsEeNJoa/international-airline-passengers.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3b0acbe60462c89f9ad9b98ca7.dl.dropboxusercontent.com/cd/0/inline/AusMuqPWgdUYc96MumGf-i9MdeJhYSWefVImDU4jUfrDgu51YCARbqcSXE1yhRjNVfUmmDvFAor7kyXkHzDgVZaMd26TxEGkSyzS0lpDGzVIYQ/file# [following]
--2019-12-22 06:07:46--  https://uc3b0acbe60462c89f9ad9b98ca7.dl.dropboxuserconten

In [0]:
data = pd.read_csv("/content/international-airline-passengers.csv?dl=0")

  

In [32]:
data.head()

,Month,passengers_in_100_thousands
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [33]:
data.shape

(144, 2)

In [0]:
ts_data = data.iloc[:,1]

In [0]:
#sns.lineplot(ts_data)

In [0]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(pd.DataFrame(data=ts_data))

In [37]:
dataset.shape

(144, 1)

In [38]:
dataset

array([[0.01544402],
       [0.02702703],
       [0.05405405],
       [0.04826255],
       [0.03281853],
       [0.05984556],
       [0.08494208],
       [0.08494208],
       [0.06177606],
       [0.02895753],
       [0.        ],
       [0.02702703],
       [0.02123552],
       [0.04247104],
       [0.07142857],
       [0.05984556],
       [0.04054054],
       [0.08687259],
       [0.12741313],
       [0.12741313],
       [0.1042471 ],
       [0.05598456],
       [0.01930502],
       [0.06949807],
       [0.07915058],
       [0.08880309],
       [0.14285714],
       [0.11389961],
       [0.13127413],
       [0.14285714],
       [0.18339768],
       [0.18339768],
       [0.15444015],
       [0.11196911],
       [0.08108108],
       [0.11969112],
       [0.12934363],
       [0.14671815],
       [0.17181467],
       [0.14864865],
       [0.15250965],
       [0.22007722],
       [0.24324324],
       [0.26640927],
       [0.2027027 ],
       [0.16795367],
       [0.13127413],
       [0.173

In [39]:
#144*0.60
train_ratio = 0.60
train_size = int(dataset.shape[0]*train_ratio)
train_size


86

In [0]:
train = dataset[:train_size,:]
test = dataset[train_size:,:]

In [41]:
print(train.shape)
print(test.shape)

(86, 1)
(58, 1)


In [0]:
def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [43]:
window_size = 10
train_X, train_Y = create_dataset(train, window_size)
test_X, test_Y = create_dataset(test, window_size)
print("Original training data shape:")
print(train_X.shape)

Original training data shape:
(75, 10)


In [44]:
pd.DataFrame(train_X).head()

,0,1,2,3,4,5,6,7,8,9
0,0.015444,0.027027,0.054054,0.048263,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958
1,0.027027,0.054054,0.048263,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958,0.000000
2,0.054054,0.048263,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958,0.000000,0.027027
3,0.048263,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958,0.000000,0.027027,0.021236
4,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958,0.000000,0.027027,0.021236,0.042471


In [45]:
# shape of data should be : (num_obs, feature_dimension, sequence_length)
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New training data shape:")
print(train_X.shape)

New training data shape:
(75, 1, 10)


In [46]:
# input shape : (feature_dimension, seq_length)
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
model = Sequential()
    
model.add(LSTM(4, 
               input_shape = (1, window_size)))
model.add(Dense(1))
model.compile(loss = "mean_squared_error", 
              optimizer = "adam")
model.fit(train_X, 
          train_Y, 
          epochs = 100, 
          batch_size = 1, 
          verbose = 2,
          validation_data = (test_X, test_Y))


es = EarlyStopping (monitor = 'val_loss', patience = 3)

Train on 75 samples, validate on 47 samples
Epoch 1/100
 - 1s - loss: 0.0303 - val_loss: 0.1644
Epoch 2/100
 - 0s - loss: 0.0093 - val_loss: 0.0837
Epoch 3/100
 - 0s - loss: 0.0066 - val_loss: 0.0617
Epoch 4/100
 - 0s - loss: 0.0060 - val_loss: 0.0526
Epoch 5/100
 - 0s - loss: 0.0056 - val_loss: 0.0442
Epoch 6/100
 - 0s - loss: 0.0052 - val_loss: 0.0364
Epoch 7/100
 - 0s - loss: 0.0049 - val_loss: 0.0330
Epoch 8/100
 - 0s - loss: 0.0046 - val_loss: 0.0306
Epoch 9/100
 - 0s - loss: 0.0044 - val_loss: 0.0301
Epoch 10/100
 - 0s - loss: 0.0043 - val_loss: 0.0251
Epoch 11/100
 - 0s - loss: 0.0041 - val_loss: 0.0252
Epoch 12/100
 - 0s - loss: 0.0040 - val_loss: 0.0228
Epoch 13/100
 - 0s - loss: 0.0038 - val_loss: 0.0204
Epoch 14/100
 - 0s - loss: 0.0037 - val_loss: 0.0189
Epoch 15/100
 - 0s - loss: 0.0035 - val_loss: 0.0170
Epoch 16/100
 - 0s - loss: 0.0035 - val_loss: 0.0175
Epoch 17/100
 - 0s - loss: 0.0034 - val_loss: 0.0173
Epoch 18/100
 - 0s - loss: 0.0033 - val_loss: 0.0156
Epoch 19/10

In [0]:
md = ModelCheckpoint(filepath = "/content/best_LSTM_model.h5", verbose = 1, save_best_only = True)

In [52]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 4)                 240       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 245
Trainable params: 245
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Make predictions on the original scale of the data.
pred = scaler.inverse_transform(model.predict(train_X))
# Prepare Y data to also be on the original scale for interpretability.
orig_data = scaler.inverse_transform([train_Y])
# Calculate RMSE.
score_train = np.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))

In [0]:
# Make predictions on the original scale of the data.
pred = scaler.inverse_transform(model.predict(test_X))
# Prepare Y data to also be on the original scale for interpretability.
orig_data = scaler.inverse_transform([test_Y])
# Calculate RMSE.
score_test = np.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))

In [55]:
print("Training data score: ",  score_train)
print("Test data score: ",  score_test)

Training data score:  18.87615913619388
Test data score:  37.67475199216503


Mutliple Time Series

In [56]:
!wget https://www.dropbox.com/sh/edjj4duz16m6fdj/AADeMAKVRqJfFCVp_U_rnCe5a/all_stocks_2006-01-01_to_2018-01-01.csv

--2019-12-22 06:07:00--  https://www.dropbox.com/sh/edjj4duz16m6fdj/AADeMAKVRqJfFCVp_U_rnCe5a/all_stocks_2006-01-01_to_2018-01-01.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/edjj4duz16m6fdj/AADeMAKVRqJfFCVp_U_rnCe5a/all_stocks_2006-01-01_to_2018-01-01.csv [following]
--2019-12-22 06:07:00--  https://www.dropbox.com/sh/raw/edjj4duz16m6fdj/AADeMAKVRqJfFCVp_U_rnCe5a/all_stocks_2006-01-01_to_2018-01-01.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc29d95b4d9904feae29aed9022a.dl.dropboxusercontent.com/cd/0/inline/AusYQb3sFdwP0sQslKX7wpinj9L9d1b7EOX_pmPymeMUK1-v2Qci2ZTvc67ry9zBb8Gul_OM2VY9BbvHUFv1Feee24slxBCqTXLMFjSKFMBu8g/file# [following]
--2019-12-22 06:07:00--  https://uc29d95b4d9904feae29aed9022a.dl.dropboxu

In [0]:
data_stocks = pd.read_csv("/content/sample_data/all_stocks_2006-01-01_to_2018-01-01.csv")

In [69]:
data_stocks.head()

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,77.76,79.35,77.24,79.11,3117200,MMM
1,2006-01-04,79.49,79.49,78.25,78.71,2558000,MMM
2,2006-01-05,78.41,78.65,77.56,77.99,2529500,MMM
3,2006-01-06,78.64,78.90,77.64,78.63,2479500,MMM
4,2006-01-09,78.50,79.83,78.46,79.02,1845600,MMM


In [70]:
data_stocks.shape

(93612, 7)

In [0]:
data = data_stocks[['Date','Close','Name']]

In [72]:
data.head()

,Date,Close,Name
0,2006-01-03,79.11,MMM
1,2006-01-04,78.71,MMM
2,2006-01-05,77.99,MMM
3,2006-01-06,78.63,MMM
4,2006-01-09,79.02,MMM


In [75]:
data.Name.value_counts()

AXP      3020
MCD      3020
JNJ      3020
TRV      3020
CAT      3020
UNH      3020
CVX      3020
MMM      3020
IBM      3020
WMT      3020
HD       3020
XOM      3020
UTX      3020
DIS      3020
BA       3020
KO       3020
GS       3020
PG       3020
PFE      3020
NKE      3020
VZ       3020
JPM      3020
GE       3020
MSFT     3019
INTC     3019
AAPL     3019
CSCO     3019
GOOGL    3019
AABA     3019
AMZN     3019
MRK      3019
Name: Name, dtype: int64

In [74]:
multi_ts = data.pivot_table(columns = "Name",values="Close",index = "Date")
multi_ts

Name,AABA,AAPL,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GE,GOOGL,GS,HD,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006-01-03,40.91,10.68,47.58,52.58,70.44,57.80,17.45,59.08,24.40,35.37,217.83,128.87,41.24,82.06,25.57,61.63,40.19,20.45,33.52,79.11,32.75,26.84,10.74,23.78,58.78,45.99,61.73,56.53,30.38,46.23,58.47
2006-01-04,40.97,10.71,47.25,51.95,71.17,59.27,17.85,58.91,23.99,35.32,222.84,127.09,40.48,81.95,25.91,62.58,39.62,20.41,33.82,78.71,33.13,26.97,10.69,24.55,58.89,46.50,61.88,56.19,31.27,46.32,58.57
2006-01-05,41.53,10.63,47.65,52.50,70.33,59.27,18.35,58.19,24.41,35.23,225.85,127.04,40.08,82.50,26.27,62.32,39.74,20.51,33.86,77.99,33.18,26.99,10.76,24.58,58.70,46.95,61.69,55.98,31.63,45.69,58.28
2006-01-06,43.21,10.90,47.87,52.68,69.35,60.45,18.77,59.25,24.74,35.47,233.06,128.84,40.38,84.95,26.31,62.60,40.02,20.70,34.06,78.63,33.12,26.91,10.72,24.85,58.64,47.21,62.90,56.16,31.35,45.88,59.43
2006-01-09,43.42,10.86,47.08,53.99,68.77,61.55,19.06,58.95,25.00,35.38,233.68,130.39,40.82,83.73,26.47,62.99,40.67,20.80,34.71,79.02,33.66,26.86,10.88,24.85,59.08,47.23,61.40,56.80,31.48,45.71,59.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-22,71.58,175.01,1168.36,98.74,295.10,155.75,38.55,124.98,108.67,17.50,1068.86,258.97,188.13,152.50,46.70,140.12,107.45,45.59,171.42,234.73,56.36,85.51,63.29,36.14,92.13,134.39,220.00,127.23,53.19,98.21,83.97
2017-12-26,69.86,170.57,1176.76,98.57,295.36,156.44,38.48,125.98,108.12,17.43,1065.85,257.72,190.36,152.83,46.08,140.09,107.02,45.81,171.29,235.45,56.34,85.40,63.65,36.21,92.48,134.78,219.60,127.14,53.22,99.16,83.98
2017-12-27,70.06,170.60,1182.26,99.13,295.62,157.52,38.56,125.55,107.64,17.38,1060.20,255.95,190.19,153.13,46.11,140.57,107.22,45.93,172.67,236.20,56.34,85.71,62.95,36.33,92.10,134.77,220.42,127.58,53.28,99.26,83.90


In [76]:
multi_ts.isnull().sum()

Name
AABA     1
AAPL     1
AMZN     1
AXP      0
BA       0
CAT      0
CSCO     1
CVX      0
DIS      0
GE       0
GOOGL    1
GS       0
HD       0
IBM      0
INTC     1
JNJ      0
JPM      0
KO       0
MCD      0
MMM      0
MRK      1
MSFT     1
NKE      0
PFE      0
PG       0
TRV      0
UNH      0
UTX      0
VZ       0
WMT      0
XOM      0
dtype: int64

In [0]:
multi_ts.fillna(0, inplace = True)

In [78]:
multi_ts.isnull().sum()

Name
AABA     0
AAPL     0
AMZN     0
AXP      0
BA       0
CAT      0
CSCO     0
CVX      0
DIS      0
GE       0
GOOGL    0
GS       0
HD       0
IBM      0
INTC     0
JNJ      0
JPM      0
KO       0
MCD      0
MMM      0
MRK      0
MSFT     0
NKE      0
PFE      0
PG       0
TRV      0
UNH      0
UTX      0
VZ       0
WMT      0
XOM      0
dtype: int64

In [79]:
multi_ts.columns

Index(['AABA', 'AAPL', 'AMZN', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GE',
       'GOOGL', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM',
       'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'VZ', 'WMT',
       'XOM'],
      dtype='object', name='Name')

In [0]:
scaler = MinMaxScaler(feature_range=(0,1))

dataset = scaler.fit_transform(multi_ts)

In [81]:
dataset

array([[0.56094886, 0.06053735, 0.03978826, ..., 0.16852146, 0.06904969,
        0.03974064],
       [0.56177156, 0.0607074 , 0.03951231, ..., 0.19682035, 0.07061901,
        0.04183225],
       [0.56945016, 0.06025394, 0.0398468 , ..., 0.20826709, 0.05963383,
        0.03576658],
       ...,
       [0.9606472 , 0.96701054, 0.98865223, ..., 0.89666137, 0.99372276,
        0.57163773],
       [0.95735637, 0.96973132, 0.99186339, ..., 0.90143084, 0.99616391,
        0.57414767],
       [0.95776772, 0.95924498, 0.97795673, ..., 0.88553259, 0.98482999,
        0.56619954]])

In [0]:
train_fraction = 0.90

train_size = int(len(dataset) * train_fraction)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, ], dataset[train_size:len(dataset), ]

In [98]:
train.shape, test.shape

((2718, 31), (302, 31))

In [0]:
def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size),]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [0]:
window_size = 10
train_X, train_Y = create_dataset(train,window_size=10)
test_X, test_Y = create_dataset(test,window_size=10)

In [101]:
train_X.shape

(2707, 10, 31)

In [102]:
train_Y.shape

(2707,)

In [103]:
train_X[0].shape

(10, 31)

the Reshape done below is different from the code present in LMS. [0] is the feature and [2] is the target and then the windows size

In [0]:
train_X = np.reshape(train_X, (train_X.shape[0],  train_X.shape[2],window_size,))
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[2], window_size,))

In [106]:
model = Sequential()
model.add(LSTM(4, input_shape=(train_X.shape[1], window_size)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_X, train_Y, epochs=3, batch_size=1, verbose=1)

Epoch 1/3
2707/2707 [==============================] - 45s 17ms/step - loss: 0.0077
Epoch 2/3
2707/2707 [==============================] - 43s 16ms/step - loss: 0.0020
Epoch 3/3
2707/2707 [==============================] - 44s 16ms/step - loss: 0.0016


In [0]:
testPredict = model.predict(test_X)

the below line of code will retransform the code back to the orginal scale as we did MinMax scaler

In [0]:
def inverse_transform(data,ymax,ymin):
    newdata=ymin+data*(ymax-ymin)
    return(newdata)

The y_min and y_max is directly searched from the target variable(AABA) column and then feed into the function prepared.

In [0]:
y_min=min(multi_ts['AABA'])
y_max=max(multi_ts['AABA'])
testPredict = inverse_transform(testPredict,y_max,y_min)
test_Y = inverse_transform(test_Y,y_max,y_min)

In [110]:
import math

testScore = math.sqrt(mean_squared_error(test_Y, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

Test Score: 9.15 RMSE
